# Model Output

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision.models import resnet18

In [ ]:
model = resnet18().eval()

In [ ]:
list(model.modules())[-1]

In [ ]:
out = model(torch.randn(1, 3, 32, 32))

In [ ]:
out.shape

In [ ]:
np.max(out.detach().numpy()), np.min(out.detach().numpy())

In [ ]:
plt.figure(figsize=(30, 10))
plt.bar(range(1000), out.detach().numpy().reshape(-1))
plt.show()

# Configuration

In [ ]:
import json
from KD_Lib.models.resnet import ResNet18, ResNet50, ResNet152

In [ ]:
class Cfg:
    def __init__(self, dict=None):
        if dict is not None:
            for key in dict:
                setattr(self, key, dict[key])
            return
        
        self.MODE: str = 'shake' # 'kd' or 'dml' or 'shake'
        self.DATASET: str = 'cifar100' # 'cifar10' or 'cifar100'
        self.CLASSES: int = 100
        self.DATA_PATH: str = '../Knowledge-Distillation-Zoo/datasets/'
        self.BATCH_SIZE: int = 128
        self.TEACHER = 'resnet152' 
        self.STUDENT = 'resnet18'
        self.LR: float = 0.1
        self.LR_MIN: float = 1e-6 #1e-5
        self.T: float = 1.0
        self.W: float = 0.5
        self.EPOCHS: int = 200
        self.SCHEDULER: str = 'cos' # 'cos' or 'step'
        self.TEACHER_WEIGHTS: str = f'./models/teacher_{self.DATASET}_{self.MODE}.pt'
        self.PARALLEL: bool = False
        self.EXP: str = f"{self.MODE}_{self.DATASET}"

In [ ]:
cfg = Cfg()
cfg.__dict__

In [ ]:
with open("cfg.json", "w") as file:
    json.dump(cfg.__dict__, file)
 
with open("cfg.json", "r") as file:
    loaded_cfg = json.load(file)
 
print(loaded_cfg)

In [ ]:
cfg = Cfg(loaded_cfg)

In [ ]:
cfg.__dict__

# Visualize Scheduler

In [ ]:
import torch
from torch.optim.lr_scheduler import CosineAnnealingLR, LinearLR, MultiStepLR
import matplotlib.pyplot as plt

In [ ]:
STEPS = 200
LR = 0.1
ETA = 1e-5

lrs = []
optimizer = torch.optim.SGD([torch.tensor(1)], lr=LR)
scheduler = CosineAnnealingLR(optimizer, STEPS, eta_min=ETA, last_epoch=-1)
for _ in range(STEPS):
    optimizer.step()
    lrs.append(scheduler.get_last_lr())
    scheduler.step()
plt.plot(lrs, label=scheduler.__class__.__name__)

lrs = []
optimizer = torch.optim.SGD([torch.tensor(1)], lr=LR)
scheduler = LinearLR(optimizer, total_iters=STEPS, start_factor=1, end_factor=ETA/LR)
for _ in range(STEPS):
    optimizer.step()
    lrs.append(scheduler.get_last_lr())
    scheduler.step()
plt.plot(lrs, label=scheduler.__class__.__name__)

lrs = []
optimizer = torch.optim.SGD([torch.tensor(1)], lr=LR)
scheduler = MultiStepLR(optimizer, [60, 120, 180], gamma=0.1)
for _ in range(STEPS):
    optimizer.step()
    lrs.append(scheduler.get_last_lr())
    scheduler.step()
plt.plot(lrs, label=scheduler.__class__.__name__)

#plt.semilogy()
plt.legend()
plt.show()

## CUB200

In [ ]:
import torch
import numpy as np
from torchvision import datasets, transforms
from KD_Lib.datasets import Cub200

DATASET = 'cub200'
DATA_PATH = '../Knowledge-Distillation-Zoo/datasets/'
BATCH_SIZE = 128

In [ ]:
if DATASET == 'cifar100':
    dataset = datasets.CIFAR100
    mean = (0.5071, 0.4865, 0.4409)
    std  = (0.2673, 0.2564, 0.2762)
    imsize = 32
elif DATASET == 'cub200':
    dataset = Cub200
    mean = (104/255.0, 117/255.0, 128/255.0)
    std = (1/255.0, 1/255.0, 1/255.0)
    imsize = 227

train_transform = transforms.Compose([
    transforms.RandomCrop(imsize, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
    ])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
    ])

trainset = dataset(root=DATA_PATH, train=True, download=False, transform=train_transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

In [ ]:
trainset[0][0].shape

In [ ]:
trainset[0][0].shape

In [ ]:
mean

## ResNet

In [ ]:
from torchvision.models import resnet18 as ResNet18, resnet50 as ResNet50, resnet152 as ResNet152

def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [ ]:
model = ResNet18(weights=None, num_classes=100)

In [ ]:
get_n_params(model)

In [ ]:
from KD_Lib.models.resnet import ResNet18, ResNet50, ResNet152

In [ ]:
model_new = ResNet18(num_classes=100)

In [ ]:
len(list(model_new.parameters()))

In [ ]:
get_n_params(model_new)

In [ ]:
model

In [ ]:
model_new

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from KD_Lib.models.resnet_torch import get_ResNet

In [ ]:
model = get_ResNet('resnet18', 10).eval()

In [ ]:
test = torch.randn(1, 3, 32, 32)

In [ ]:
model.forward(test, norm_feats=False)

In [ ]:
np.hist(model.forward(test, norm_feats=True))

In [ ]:
out, feats, weight, bias = model.forward(torch.randn(2, 3, 32, 32), return_feats=True)

# Track Experiments

In [2]:
import os
import json
import pandas as pd

In [3]:
DIR = './exp/'

experiments = []
for dirname, _, filenames in os.walk(DIR):
    for filename in filenames:
        experiments.append(json.load(open(os.path.join(dirname, filename), 'r')))

df = pd.DataFrame(experiments)
T = df[df['EXP'] == 'kd_cifar100_new'].iloc[0]['TIME']


In [4]:
T

115.13368615706761

In [5]:
# CRD
T_KD = 109.26
T_CRD = 156.58
T_CRD / T_KD

1.433095368844957

In [6]:
df['T_LAST'] = df['VACC'].apply(lambda x: x['T_LAST'])
df['T_BEST'] = df['VACC'].apply(lambda x: x['T_BEST'])
df['S_LAST'] = df['VACC'].apply(lambda x: x['S_LAST'])
df['S_BEST'] = df['VACC'].apply(lambda x: x['S_BEST'])
df['TIME'] = df['TIME'].apply(lambda x: x/T)
df[['EXP', 'T', 'W', 'FEAT_NORM', 'T_LAST', 'T_BEST', 'S_LAST', 'S_BEST', 'TIME']].sort_values(by='S_BEST', ascending=False)

,EXP,T,W,FEAT_NORM,T_LAST,T_BEST,S_LAST,S_BEST,TIME
15,shake_cifar100_new,4.0,1.0,False,0.7444,0.7448,0.7133,0.7160,1.110229e+00
9,dml_cifar100_T,4.0,1.0,False,0.7609,0.7622,0.7111,0.7140,NaN
1,smooth_cifar100_noCE,4.0,1.0,False,0.7432,0.7432,0.7110,0.7123,1.017295e+00
14,ftkd_cifar100_T,4.0,1.0,False,0.7396,0.7373,0.7118,0.7121,NaN
6,smooth_cifar100_new,4.0,1.0,False,0.7438,0.7436,0.7096,0.7108,1.015161e+00
8,smooth_cifar100_untrained,4.0,1.0,False,0.7428,0.7428,0.7102,0.7102,1.040369e+00
18,smooth_cifar100_no_mse,4.0,1.0,False,0.7438,0.7439,0.7062,0.7096,NaN
7,trikd_cifar100_T,4.0,1.0,False,0.7321,0.7321,0.7081,0.7091,1.072123e+00
0,dml_cifar100_new,4.0,1.0,False,0.7558,0.7586,0.7018,0.7082,2.722962e+00
10,smooth_cifar100,4.0,1.0,False,0.7437,0.7434,0.7077,0.7077,NaN


# Model Parameters

In [ ]:
from KD_Lib.models import model_dict
from KD_Lib.models.resnet_torch import monkey_patch

In [ ]:
model = model_dict['resnet20'](num_classes=100)
model = monkey_patch(model, custom=True)
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(f"Parameters: {pytorch_total_params}")

In [ ]:
from KD_Lib.models.resnet_test import ResNet50
from KD_Lib.models.shake import ShakeHead

In [ ]:
model = ResNet50(num_classes=1000)

In [ ]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(f"Parameters: {pytorch_total_params}")

In [ ]:
data = torch.randn(2, 3, 224, 224)
feat_t, out_t = model(data, is_feat=True)
shake = ShakeHead(feat_t)

In [ ]:
pytorch_total_params = sum(p.numel() for p in shake.parameters())
print(f"Parameters: {pytorch_total_params}")

# Distillation Loss

In [ ]:
import torch

In [ ]:
label = torch.randint(0, 10, (1,))
pred_t, pred_f = torch.zeros(10), torch.zeros(10)
pred_t[label] = 0.8
pred_t += 0.2 / 10
pred_f[torch.randint(0, 10, (1,))] = 0.8
pred_f += 0.2 / 10
pred_t_s = torch.softmax(pred_t/4, dim=-1)
pred_f_s = torch.softmax(pred_f/4, dim=-1)

In [ ]:
pred_t, pred_f, pred_t_s, pred_f_s

In [ ]:
torch.nn.functional.cross_entropy(pred_t.unsqueeze(0), label), torch.nn.functional.cross_entropy(pred_f.unsqueeze(0), label)

In [ ]:
torch.nn.functional.cross_entropy(pred_t_s.unsqueeze(0), label), torch.nn.functional.cross_entropy(pred_f_s.unsqueeze(0), label)